In [48]:
import pandas as pd
import numpy as np
import neurokit2 as nk
import matplotlib.pyplot as plt
from te import transfer_entropy
from cjte import calculate_cjte

In [49]:
def calc_rr_sap(
    signal: np.array,
    sampling_rate: int = 200,
    mindelay: float = 0.3,
) -> dict[np.array]:
    # Fill missing values
    signal = nk.signal_fillmissing(signal)
    cleaned_signal = nk.ppg_clean(signal, sampling_rate=sampling_rate, method="elgendi")
    peaks_info = nk.ppg_findpeaks(
        cleaned_signal, sampling_rate=sampling_rate, method="elgendi", mindelay=mindelay
    )
    peaks = peaks_info["PPG_Peaks"]
    rr = np.diff(peaks) / sampling_rate * 1000
    sap = np.array([signal[peak] for peak in peaks])[1:]
    return {"HP": 1 / rr * 1000, "SAP": sap}

In [63]:
def calc_map(    
    signal: np.array,
    sampling_rate: int = 200,
    mindelay: float = 0.3,):
    # Fill missing values
    signal = nk.signal_fillmissing(signal)
    cleaned_signal = nk.ppg_clean(signal, sampling_rate=sampling_rate, method="elgendi")
    peaks_max = nk.ppg_findpeaks(
            cleaned_signal, sampling_rate=sampling_rate, method="elgendi", mindelay=mindelay
        )["PPG_Peaks"]
    peaks_min = nk.ppg_findpeaks(
            cleaned_signal*-1, sampling_rate=sampling_rate, method="elgendi", mindelay=mindelay
        )["PPG_Peaks"]
    
    peaks = np.sort(np.concatenate((peaks_max, peaks_min)))
    first_peak = 0 if peaks[0] < peaks[1] else 1

    # MAP = (2*DP + SP) / 3
    map_ = []
    for i in range(first_peak, len(peaks)-2, 2):
        dp = signal[peaks[i]]
        sp = signal[peaks[i+1]]
        map_.append((2*dp + sp) / 3)
    
    return np.array(map_)

In [77]:
valid_datasets = [i for i in range(1, 38) if i not in [3, 7, 14, 15, 10, 11, 17, 27, 35]]

location = lambda on: f'data/CONTROL_BREATHING_RECORDINGS/OCHOTNIK_{on}/CLEAN_DATA_15_BREATHS/OCH_{on}_CLEAN_CB_15.csv'
for i in valid_datasets:
    abp_signal = pd.read_csv(location(i), sep=';', decimal=',')['abp_cnap[mmHg]'].values
    map_ = calc_map(abp_signal)
    print(len(map_))

338
356
154
259
373
366
445
267
189
243
299
201
348
222
370
370
499
377
451
222
350
234
256
266
276
335
367
488


In [52]:
def calculate_mcbfv(    
    signal: np.array,
    sampling_rate: int = 200,
    mindelay: float = 0.3):
    # Fill missing values
    signal = nk.signal_fillmissing(signal)
    cleaned_signal = nk.ppg_clean(signal, sampling_rate=sampling_rate, method="elgendi") *-1 # invert signal to find minima
    peaks_info = nk.ppg_findpeaks(
            cleaned_signal, sampling_rate=sampling_rate, method="elgendi", mindelay=mindelay
        )
    peaks = peaks_info["PPG_Peaks"]

    mcbfv = np.array([np.mean(signal[peaks[i-1]:peaks[i]]) for i, _ in enumerate(peaks) if i > 0])
    return mcbfv

In [53]:
def plot_signals(signals):
    fig, axs = plt.subplots(5, 1, figsize=(20, 10))
    for i, (key, value) in enumerate(signals.items()):
        axs[i].plot(value)
        axs[i].set_title(key)
    plt.show()

In [54]:
def calculate_te(signals):
    te = {}
    te['HP->SAP'] = transfer_entropy(signals["HP"], signals["SAP"])
    te['SAP->HP'] = transfer_entropy(signals["SAP"], signals["HP"])
    # adjust lenght of MAP and MCBFV
    min_len = min(len(signals["MAP"]), len(signals["MCBFV"]))
    signals["MAP"] = signals["MAP"][:min_len]
    signals["MCBFV"] = signals["MCBFV"][:min_len]
    te['MCBFV->MAP'] = transfer_entropy(signals["MCBFV"], signals["MAP"])
    te['MAP->MCBFV'] = transfer_entropy(signals["MAP"], signals["MCBFV"])
    return te

In [55]:
def conditional_joint_transfer_entropy(signals):
    cjte = {}
    r_widnowed = np.array_split(signals["R"], len(signals["HP"]))
    signalsr_matched = np.array([np.mean(window) for window in r_widnowed])
    cjte['HP->SAP'] = calculate_cjte(signals["HP"], signals["SAP"], signalsr_matched)
    cjte['SAP->HP'] = calculate_cjte(signals["SAP"], signals["HP"], signalsr_matched)

    min_len = min(len(signals["MAP"]), len(signals["MCBFV"]))
    signals["MAP"] = signals["MAP"][:min_len]
    signals["MCBFV"] = signals["MCBFV"][:min_len]
    r_widnowed = np.array_split(signals["R"], len(signals["MAP"]))
    signalsr_matched = np.array([np.mean(window) for window in r_widnowed])
    cjte['MCBFV->MAP'] = calculate_cjte(signals["MCBFV"], signals["MAP"], signalsr_matched)
    cjte['MAP->MCBFV'] = calculate_cjte(signals["MAP"], signals["MCBFV"], signalsr_matched)
    return cjte

In [56]:
def plot_results(data, title):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_axes([0, 0, 1, 1])
    bp = ax.boxplot(data)
    ax.set_title(title)
    ax.set_xticklabels(['BASELINE', '6B', '10B', '15B'])
    return fig

In [57]:
def read_fv(location):
    possible_fv_names = ['fvl', 'fvr']
    fvl = pd.read_csv(location, sep=';', decimal=',')['fvl'].values
    fvr = pd.read_csv(location, sep=';', decimal=',')['fvr'].values
    fvl_mean = np.mean(fvl)
    fvr_mean = np.mean(fvr)

    if fvl_mean > 30:
        return fvl
    elif fvr_mean > 30:
        return fvr
    else:
        return None


In [58]:
valid_datasets = [i for i in range(1, 38) if i not in [3, 7, 14, 15, 10, 11, 17, 27, 35]]

location_baseline = lambda on: f'data/CONTROL_BREATHING_RECORDINGS/OCHOTNIK_{on}/CLEAN_DATA_BASELINE/OCH_{on}_CLEAN_CB_BASELINE.csv'
location_6 = lambda on: f'data/CONTROL_BREATHING_RECORDINGS/OCHOTNIK_{on}/CLEAN_DATA_6_BREATHS/OCH_{on}_CLEAN_CB_6.csv'
location_10 = lambda on: f'data/CONTROL_BREATHING_RECORDINGS/OCHOTNIK_{on}/CLEAN_DATA_10_BREATHS/OCH_{on}_CLEAN_CB_10.csv'
location_15 = lambda on: f'data/CONTROL_BREATHING_RECORDINGS/OCHOTNIK_{on}/CLEAN_DATA_15_BREATHS/OCH_{on}_CLEAN_CB_15.csv'
rc_hp_sap = []
rc_sap_hp = []
rc_mcbfv_map = []
rc_map_mcbfv = []
rc_hp_sap_cjte = []
rc_sap_hp_cjte = []
rc_mcbfv_map_cjte = []
rc_map_mcbfv_cjte = []
for loc in [location_baseline, location_6, location_10, location_15]:
    r_hp_sap = []
    r_sap_hp = []
    r_mcbfv_map = []
    r_map_mcbfv = []
    r_hp_sap_cjte = []
    r_sap_hp_cjte = []
    r_mcbfv_map_cjte = []
    r_map_mcbfv_cjte = []
    for on in valid_datasets:
        location = loc(on)
        abp_signal = pd.read_csv(location, sep=';', decimal=',')['abp_cnap[mmHg]'].values
        fv = read_fv(location)
        if fv is None:
            continue
        r = pd.read_csv(location, sep=';', decimal=',')['rr[rpm]'].values
        signals = calc_rr_sap(abp_signal)
        signals['MAP'] = calc_map(abp_signal)
        signals['MCBFV'] = calculate_mcbfv(fv)
        signals['R'] = r

        #plot_signals(signals)
        #print(location)

        results = calculate_te(signals)
        r_hp_sap.append(results['HP->SAP'])
        r_sap_hp.append(results['SAP->HP'])
        r_mcbfv_map.append(results['MCBFV->MAP'])
        r_map_mcbfv.append(results['MAP->MCBFV'])
        results_cjte = conditional_joint_transfer_entropy(signals)
        r_hp_sap_cjte.append(results_cjte['HP->SAP'])
        r_sap_hp_cjte.append(results_cjte['SAP->HP'])
        r_mcbfv_map_cjte.append(results_cjte['MCBFV->MAP'])
        r_map_mcbfv_cjte.append(results_cjte['MAP->MCBFV'])
    rc_hp_sap.append(r_hp_sap)
    rc_sap_hp.append(r_sap_hp)
    rc_mcbfv_map.append(r_mcbfv_map)
    rc_map_mcbfv.append(r_map_mcbfv)
    rc_hp_sap_cjte.append(r_hp_sap_cjte)
    rc_sap_hp_cjte.append(r_sap_hp_cjte)
    rc_mcbfv_map_cjte.append(r_mcbfv_map_cjte)
    rc_map_mcbfv_cjte.append(r_map_mcbfv_cjte)

#fig_1 = plot_results(rc_hp_sap, 'HP->SAP')
#fig_2 = plot_results(rc_sap_hp, 'SAP->HP')
#fig_3 = plot_results(rc_mcbfv_map, 'MCBFV->MAP')
#fig_4 = plot_results(rc_map_mcbfv, 'MAP->MCBFV')
# make 1 plot with 4 subplots from my figures above
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs[0, 0].boxplot(rc_hp_sap)
axs[0, 0].set_title('TE(HP->SAP)')
axs[0, 1].boxplot(rc_sap_hp)
axs[0, 1].set_title('TE(SAP->HP)')
axs[1, 0].boxplot(rc_mcbfv_map)
axs[1, 0].set_title('TE(MCBFV->MAP)')
axs[1, 1].boxplot(rc_map_mcbfv)
axs[1, 1].set_title('TE(MAP->MCBFV)')
for i in range(2):
    for j in range(2):
        axs[i, j].set_xticklabels(['BASELINE', '6B', '10B', '15B'])
plt.show()

fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs[0, 0].boxplot(rc_hp_sap_cjte)
axs[0, 0].set_title('CJTE(HP->SAP)')
axs[0, 1].boxplot(rc_sap_hp_cjte)
axs[0, 1].set_title('CJTE(SAP->HP)')
axs[1, 0].boxplot(rc_mcbfv_map_cjte)
axs[1, 0].set_title('CJTE(MCBFV->MAP)')
axs[1, 1].boxplot(rc_map_mcbfv_cjte)
axs[1, 1].set_title('CJTE(MAP->MCBFV)')
for i in range(2):
    for j in range(2):
        axs[i, j].set_xticklabels(['BASELINE', '6B', '10B', '15B'])
plt.show()



c:\Users\Ignacy\Desktop\Brainlab_TE\cjte.py:19: RuntimeWarning: invalid value encountered in divide
  return (signal - np.mean(signal)) / np.std(signal)


ValueError: array must not contain infs or NaNs